In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/monthly-food-price-estimates/WLD_RTP_details_2023-10-02.csv
/kaggle/input/monthly-food-price-estimates/ddi-documentation-english_microdata-4483 (1).pdf
/kaggle/input/monthly-food-price-estimates/ddi-documentation-english_microdata-4509.pdf
/kaggle/input/monthly-food-price-estimates/WLD_RTFP_country_2023-10-02.csv
/kaggle/input/spark4ai-prediction/soil_dataset/sample_submission.csv
/kaggle/input/spark4ai-prediction/soil_dataset/train.csv
/kaggle/input/spark4ai-prediction/soil_dataset/test.csv


In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
df=pd.read_csv('/kaggle/input/spark4ai-prediction/soil_dataset/train.csv')
df_part1 = df.iloc[:, :23]
df_part2=df.iloc[:,23:]
df_part1.drop('Id',axis=1)
fd=df.head(101)


In [3]:
from sklearn.metrics import mean_squared_error
model=RandomForestRegressor()
Y=df_part1[['Property_A', 'Property_B', 'Property_C', 'Property_D',
       'Property_E', 'Property_F']]

X=df_part1.drop(['Property_A', 'Property_B', 'Property_C', 'Property_D',
       'Property_E', 'Property_F','Id'],axis=1)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.401, random_state=42) 
model.fit(X_train, Y_train)
y_pred= model.predict(X_test)
y_pred1=[]
for i in range(101):
    a=0
    for j in range(6):
        a+=y_pred[i][j]
    y_pred1.append(a)    
predict4=pd.DataFrame({'Id':fd['Id'],'Predicted':y_pred1})
rmse = np.sqrt(mean_squared_error(Y_test, y_pred))
print(f"Root Mean Squared Error (RMSE): {rmse}")




Root Mean Squared Error (RMSE): 5.6304035329446736


In [4]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
model = xgb.XGBRegressor()
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)

mae = mean_absolute_error(Y_test, y_pred)
print(f"Mean Absolute Error: {mae}")
y_pred1=[]
for i in range(101):
    a=0
    for j in range(6):
        a+=y_pred[i][j]
    y_pred1.append(a)    
predict=pd.DataFrame({'Id':fd['Id'],'Predicted':y_pred1})
rmse = np.sqrt(mean_squared_error(Y_test, y_pred))
print(f"Root Mean Squared Error (RMSE): {rmse}")
        


Mean Absolute Error: 2.3349042967197113
Root Mean Squared Error (RMSE): 6.5580801572483916


In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
model = LinearRegression()
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(Y_test, y_pred))
print(f"Root Mean Squared Error (RMSE): {rmse}")


Root Mean Squared Error (RMSE): 6.876219155962303


In [6]:
X1=df_part2
model=RandomForestRegressor()
Y1=df_part1[['Property_A', 'Property_B', 'Property_C', 'Property_D',
       'Property_E', 'Property_F']]
X1_train, X1_test, Y1_train, Y1_test = train_test_split(X1,Y1, test_size=0.401, random_state=42) 
model.fit(X1_train, Y1_train)
predictions = model.predict(X1_test)
y_pred2=[]
for i in range(101):
    a=0
    for j in range(6):
        a+=predictions[i][j]
    y_pred2.append(a)    
predict2=pd.DataFrame({'Id':fd['Id'],'Predicted':y_pred1})
rmse = np.sqrt(mean_squared_error(Y1_test,predictions))
print(f"Root Mean Squared Error (RMSE): {rmse}")




Root Mean Squared Error (RMSE): 6.61615600885313


In [7]:
model = xgb.XGBRegressor()
model.fit(X1_train, Y1_train)
predictions = model.predict(X1_test)
y_pred2=[]
for i in range(101):
    a=0
    for j in range(6):
        a+=predictions[i][j]
    y_pred2.append(a)    
predict2=pd.DataFrame({'Id':fd['Id'],'Predicted':y_pred1})
rmse = np.sqrt(mean_squared_error(Y1_test,predictions))
print(f"Root Mean Squared Error (RMSE): {rmse}")


Root Mean Squared Error (RMSE): 6.8833363000768575


In [8]:
model = LinearRegression()
model.fit(X1_train, Y1_train)
predictions = model.predict(X1_test)
y_pred2=[]
for i in range(101):
    a=0
    for j in range(6):
        a+=predictions[i][j]
    y_pred2.append(a)    
predict3=pd.DataFrame({'Id':fd['Id'],'Predicted':y_pred1})
rmse = np.sqrt(mean_squared_error(Y1_test,predictions))
print(f"Root Mean Squared Error (RMSE): {rmse}")
predict3


Root Mean Squared Error (RMSE): 5.869515250532215


,Id,Predicted
0,1,108.251050
1,2,104.419890
2,3,111.975738
3,4,106.258805
4,5,100.942645
...,...,...
96,97,103.399695
97,98,109.673236
98,99,111.694479
99,100,109.899300


In [9]:
bias_1 = 1.97
bias_2 = 0.03
fused_data = (bias_1 * predict3 + bias_2 * predict4) / (bias_1 + bias_2)
fused_data['Id']=predict3['Id']
fused_data




,Id,Predicted
0,1,108.249832
1,2,104.468702
2,3,111.921062
3,4,106.288371
4,5,101.050534
...,...,...
96,97,103.471265
97,98,109.656366
98,99,111.646573
99,100,109.874449


In [10]:
csv_file_path = ' my_dataframe.csv' 
fused_data.to_csv(csv_file_path, index=False)